# Headline Cleaning Part 2: Tidying the Data
## Solution
---
There are lots of ways to do things like this in Python. If you're getting good answers, don't assume there's anything wrong with your methods.

---

Now that we're familiar with the headline data, we're ready to _really_ clean this data up.

In this workbook, we're going to make two output files. Here's what should go in them:
* `clean_headlines.txt`: This should be a tab-delimited text file with a column for the paper, the date (in `YYYY-MM-DD` format), and the text of the headline. 
* `headline_word_count.txt`: This file should be tab-delimited with columns for the paper, the date (same format as above), each word the paper used on that date, and the count of those words. The words are defined as alphanumeric characters separated by whitespace from other characters, should have punctuation removed, and should be cast to lowercase. For example, "Mr. Bean" would be two words, "mr" and "bean". 

Let's work through an example so you can see what I mean. Here's a list of the headlines for the _Missoulian_ for 2015-09-23:
* After EWU air raid, Bobcats switch focus to Cal Poly option
* Alternative art gallery FrontierSpace to hold annual art auction
* Does state gets passing grade for education funding? Group aims to find out
* Fall films have issues
* Family in need after Bonner fire destroyed home, killed cat
* Headwaters Dance Company's "last hurrah" concerts are next week
* Montana coaching tree gathers for fundraiser
* Mountain Line to detour Saturday due to UM Homecoming Parade
* Rabid bat found at picnic area north of Helena
* Ravalli County attorney's daughter jailed in sibling assault
* Search continues today for missing Kalispell bow hunter
* Tester to escort Pope Francis to House chambers

In `clean_headlines.txt` we'd expect to see a row that looks like this:
Missoulian    2015-09-23    After EWU air raid, Bobcats switch focus to Cal Poly option

In `headline_word_count.txt` we'd expect to see a row that looks like this:
Missoulian    2015-09-23    after    2

(Note that the gaps between fields should be a tab.)


In [ ]:
from collections import defaultdict
from pprint import pprint
from string import punctuation


working_dir =  "C:\\Users\\jchan\\Dropbox\\Teaching\\AppliedDataAnalytics\\Code\\headline-cleaning\\"

input_files = ["missoula.txt","sidney.txt","butte.txt","bozeman.txt","billings.txt"]
paper_names = ["Missoulian","Sidney Herald","Montana Standard","Bozeman Daily Chronicle","Billings Gazette"]

Let's open up one of these files and see what we have. 

In [ ]:
this_input = input_files[0]

with open(working_dir + this_input) as infile :
    for idx,row in enumerate(infile.readlines()) :
        print(row)
        if idx > 2 :
            break

Still a mess. I'm going to write a function that takes the paper in and builds a well-formatted date from these ugly dates. 

First, I'm just going to read in all the dates and keep them in order so that I can see what's going on. This step could be done in Excel too.

In [ ]:
dates = defaultdict(list) # paper keying a list of dates

for this_input in input_files : # notice, by not hardcoding the input file I can quickly wrap the above cell in a `for`
    with open(working_dir + this_input,'r',encoding="Latin-1") as infile :
        
        these_dates = infile.readline().strip().split("\t")
        paper = paper_names[input_files.index(this_input)] # note the use of index here. 
        
        dates[paper] = these_dates


In [ ]:
# Now I'm going to print the paper and the dates in "clean" fashion 
# to try to figure out where the 2015/2016 break is for each paper.
for paper in dates :
    print(paper + " : " + " ".join(dates[paper]))
    print("\n\n")
    
# Missoulian: 23-Sep to 30-Dec are 2015. Clean split
# Sidney: 22-Sep to 23-Dec are 2015. Clean split, barely, as 2016-09-21 is in there.
# MT Standard: 22-Sep to 29-Dec are 2015. Clean split, barely, as 2016-09-21 is in there. 
# Bozeman: 23-Sep to 30-Dec are 2015. *NOT CLEAN SPLIT*. 28-Sep and 5-Oct are special cases of 2016.
# Billings: 22-Sep to 30-Dec are 2015. Clean split, barely, as 2016-09-21 is in there. 

Okay, this is a mess. The first row has all the dates, the subsequent rows have headlines with many blanks put in. Let's start by getting all the dates. We have to read all the files because we don't know if there's a date that just appears in one of those.

In [ ]:
dates = set() # make it a set, so we don't have to keep track duplicates

for this_input in input_files : # notice, by not hardcoding the input file I can quickly wrap the above cell in a `for`
    with open(working_dir + this_input) as infile :
        for idx,row in enumerate(infile.readlines()) :
            these_dates = row.strip().split("\t")
            for a_date in these_dates :
                dates.add(a_date)
            break # We can get out after just the first row.
pprint(dates) # discovered a few errors when I did this the first time. Had the break in the wrong spot, for instance

Okay, now we're in a position to write a function that takes an ugly date and makes a pretty one. We're going to need the paper as an input into the function.

In [ ]:
def reformat_date(ugly_date, the_paper) :
    '''
        Takes as input a date of the form "D-MMM" and returns a date of the form
        "YYYY-MM-DD". Note that we have to do some work on years. Dates in Oct, Nov,
        Dec are *almost* all 2015. 
        
        For every paper other than Bozeman Daily Chronicle, dates in a year
        later than 21-Sep are in 2015. For BZN, 28-Sep and 5-Oct are in 2016.
    '''
    month_abbr = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    month_num = [n+1 for n in range(12)]
    month_str = ["{:02d}".format(n) for n in month_num]
    month_lu = dict(zip(month_abbr,month_str))
    
    d,m = ugly_date.split("-")
    m_num = month_lu[m]
    d_str = "{:02d}".format(int(d))
    
    if ((m == "Sep" and int(d) >= 22) or
        m in ["Oct","Nov","Dec"]) :
        y = "2015"
    else :
        y = "2016" 

    # Fix the BZN special cases. 
    if the_paper == "Bozeman Daily Chronicle" :
        if ((m == "Sep" and d_str == "28") or 
            (m == "Oct" and d_str == "05")) :
            y = "2016"

    pretty_date = "-".join([y,m_num,d_str])
    
    return(pretty_date)
    

In [ ]:
# Lots of testing here to make sure I've got what I expect.
assert("2015-09-23" == reformat_date("23-Sep","Missoulian"))
assert("2015-09-30" == reformat_date("30-Sep","Missoulian"))
assert("2015-12-23" == reformat_date("23-Dec","Missoulian"))
assert("2016-01-06" == reformat_date("6-Jan","Missoulian"))
assert("2016-03-09" == reformat_date("9-Mar","Missoulian"))
assert("2016-06-01" == reformat_date("1-Jun","Missoulian"))
assert("2016-09-21" == reformat_date("21-Sep","Missoulian"))

assert("2015-09-22" == reformat_date("22-Sep","Sidney Herald"))
assert("2015-09-30" == reformat_date("30-Sep","Sidney Herald"))
assert("2016-06-01" == reformat_date("1-Jun","Sidney Herald"))
assert("2016-09-21" == reformat_date("21-Sep","Sidney Herald"))

assert("2015-09-22" == reformat_date("22-Sep","Montana Standard"))
assert("2015-09-30" == reformat_date("30-Sep","Montana Standard"))
assert("2015-10-28" == reformat_date("28-Oct","Montana Standard"))
assert("2016-06-01" == reformat_date("1-Jun","Montana Standard"))
assert("2016-09-21" == reformat_date("21-Sep","Montana Standard"))

assert("2015-09-22" == reformat_date("22-Sep","Billings Gazette"))
assert("2015-09-30" == reformat_date("30-Sep","Billings Gazette"))
assert("2015-10-28" == reformat_date("28-Oct","Billings Gazette"))
assert("2016-06-01" == reformat_date("1-Jun","Billings Gazette"))
assert("2016-09-21" == reformat_date("21-Sep","Billings Gazette"))

# Bozeman: Standard ones plus special cases
assert("2015-09-23" == reformat_date("23-Sep","Bozeman Daily Chronicle"))
assert("2015-09-30" == reformat_date("30-Sep","Bozeman Daily Chronicle"))
assert("2015-10-28" == reformat_date("28-Oct","Bozeman Daily Chronicle"))
assert("2016-06-01" == reformat_date("1-Jun","Bozeman Daily Chronicle"))
assert("2016-09-21" == reformat_date("21-Sep","Bozeman Daily Chronicle"))

assert("2016-09-21" == reformat_date("21-Sep","Bozeman Daily Chronicle"))
assert("2016-09-28" == reformat_date("28-Sep","Bozeman Daily Chronicle"))
assert("2016-10-05" == reformat_date("5-Oct","Bozeman Daily Chronicle"))

Whew, that sucked. Okay, now let's start trying to parse these files. We'll fill up a default dictionary with the paper, the date, and then a list that has all the non-blank headlines. 

In [ ]:
headline_holder = defaultdict(lambda: defaultdict(list)) # If you haven't seen this before, ask me in class. Worth discussing.
hl_counter = 0

for idx,this_input in enumerate(input_files) :     
    with open(working_dir + this_input,'r',encoding="Latin-1") as infile :
        dates = infile.readline().strip().split("\t")
        this_paper = paper_names[idx]
        this_file_date_mapper = dict()
        
        for a_date in dates :
            this_file_date_mapper[a_date] = reformat_date(a_date,this_paper)

        # Now we've got a lookup so we can get to the real dates.
        for headline_row in infile.readlines() :
            potential_headlines = headline_row.strip().split("\t")

            for idx, ph in enumerate(potential_headlines) : # ph stands for potential headline
                if ph != "" : # gotta avoid the blanks
                    this_ugly_date = dates[idx]
                    this_nice_date = this_file_date_mapper[this_ugly_date]
                    headline_holder[this_paper][this_nice_date].append(ph)
                    hl_counter += 1
print("Headlines Captured: ",hl_counter)

In [ ]:
with open(working_dir + "clean_headlines.txt",'w',encoding="UTF-8") as ofile :
    ofile.write("\t".join(["paper","date","headline"])+"\n")

    for paper in headline_holder :
        for hl_date in headline_holder[paper] :
            for hl in headline_holder[paper][hl_date] :
                ofile.write("\t".join([paper, str(hl_date), hl])+"\n")

This gets the headlines cleaned, which was a big enough chore. Now we want to clean up the headlines and do word counts. Our output data set will have paper, date, word, and count. Python makes this shockingly easy. First, let's write a function that takes a headline and returns a list of the words in lowercase with all the punctuation removed.

In [ ]:
hl = "A headline test."
hl = hl.lower()

clean_hl = []
for ch in hl :
    if ch not in excluded_chars :
        clean_hl.append(ch)
        
print("|".join(clean_hl))
#print(clean_hl)

In [ ]:
excluded_chars = set(punctuation)

for ch in ["`","‘","’"] :
    excluded_chars.add(ch)

#excluded_chars.add(["`","‘","’"]) # Thanks Levi for catching these marks

def clean_headline(hl) :
    ''' Removes punctuation, changes to lowercase, and splits on whitespace.
        Returns a list of the words in a headline'''
    
    hl = hl.lower()
    hl = ''.join([ch for ch in hl if ch not in excluded_chars])
    return(hl.split())

#clean_headline("This'll `test the headline    cleaner, I think 39393")

In [ ]:
# And, as always, let's test.
assert(clean_headline("A headline test.") == ['a','headline','test'])
assert(clean_headline("Moar testin'.") == ['moar','testin'])
assert(clean_headline("Bobcats' woes.") == ['bobcats','woes'])

In [ ]:
word_count = defaultdict(
    lambda: defaultdict(
    lambda: defaultdict(int))) # Now we need one more level.

words = set()

for paper in headline_holder :
    for hl_date in headline_holder[paper] :
        for hl in headline_holder[paper][hl_date] :
            word_list = clean_headline(hl)

            for word in word_list:
                words.add(word)
                word_count[paper][hl_date][word] += 1

print("Processed ",str(len(words))," words.")

In [ ]:
with open(working_dir + "headline_word_count.txt",'w',encoding="UTF-8") as ofile :
    ofile.write("\t".join(["paper","date","word","count"])+"\n")

    for paper in word_count :
        for hl_date in word_count[paper] :
            for word in word_count[paper][hl_date] :
                ofile.write("\t".join([paper, hl_date,
                                       word,
                                       str(word_count[paper][hl_date][word])]) + "\n")